In [3]:
# Paquetes del sistema necesarios para OpenGL, video y entorno virtual
!apt-get install -y xvfb
!pip install pyopengl pyopengl-accelerate imageio pyvirtualdisplay

In [1]:
# agent_monster.py
import random
from typing import Tuple, Optional, List, Dict, Any


class AgenteReflejoMonstruo:
    """
    Agente reflejo simple que representa un monstruo energético dentro del entorno N³.

    Actúa de forma reactiva sin memoria ni razonamiento, moviéndose aleatoriamente hacia
    una Zona Libre adyacente con probabilidad `p_movimiento` en cada ciclo múltiplo de `K`.
    """

    _DIRECCIONES: Dict[str, Tuple[int, int, int]] = {
        '+X': (1, 0, 0), '-X': (-1, 0, 0),
        '+Y': (0, 1, 0), '-Y': (0, -1, 0),
        '+Z': (0, 0, 1), '-Z': (0, 0, -1)
    }

    def __init__(self, id: int, x: int, y: int, z: int, p_movimiento: float = 0.7) -> None:
        """Inicializa el agente reflejo con posición inicial y probabilidad de movimiento."""
        self.id = id
        self.x, self.y, self.z = int(x), int(y), int(z)
        self.p_movimiento = p_movimiento
        self.activo = True

    def percibir(self, entorno: Any) -> Dict[str, Any]:
        """Obtiene las direcciones válidas de movimiento hacia Zonas Libres adyacentes."""
        movimientos_validos = self._obtener_movimientos_validos(entorno)
        return {
            'movimientos_validos': movimientos_validos,
            'puede_moverse': bool(movimientos_validos)
        }

    def _obtener_movimientos_validos(self, entorno: Any) -> List[Tuple[int, int, int]]:
        """Devuelve las direcciones transitables hacia Zonas Libres dentro del entorno."""
        movimientos_validos = []
        for _, (dx, dy, dz) in self._DIRECCIONES.items():
            nx, ny, nz = self.x + dx, self.y + dy, self.z + dz
            if self._es_movimiento_valido(entorno, nx, ny, nz):
                movimientos_validos.append((dx, dy, dz))
        return movimientos_validos

    def _es_movimiento_valido(self, entorno: Any, x: int, y: int, z: int) -> bool:
        """Retorna True si la coordenada destino está dentro del entorno y es Zona Libre."""
        if not (0 <= x < entorno.N and 0 <= y < entorno.N and 0 <= z < entorno.N):
            return False
        return entorno.grid[x, y, z] != entorno.ZONA_VACIA

    def decidir_accion(self, percepcion: Dict[str, Any], ciclo_actual: int, K: int) -> Dict[str, Any]:
        """
        Determina la acción a ejecutar según el ciclo actual, la probabilidad y las zonas libres.
        """
        if ciclo_actual % K != 0:
            return {"accion": "inactivo", "direccion": None, "razon": "no_en_ciclo"}
        if random.random() > self.p_movimiento:
            return {"accion": "inactivo", "direccion": None, "razon": "no_supera_probabilidad"}
        if not percepcion["puede_moverse"]:
            return {"accion": "inactivo", "direccion": None, "razon": "sin_movimientos_validos"}

        direccion = random.choice(percepcion["movimientos_validos"])
        return {"accion": "mover", "direccion": direccion, "razon": "movimiento_aleatorio"}

    def ejecutar_accion(self, accion: str, direccion: Optional[Tuple[int, int, int]]) -> bool:
        """Ejecuta el movimiento actualizando la posición si la acción es 'mover'."""
        if accion == "mover" and direccion:
            dx, dy, dz = direccion
            self.x += dx
            self.y += dy
            self.z += dz
            return True
        return False

    def percibir_decidir_actuar(self, t: int, entorno: Any, K: int) -> Dict[str, Any]:
        """Ejecuta el ciclo completo de percepción, decisión y acción."""
        percepcion = self.percibir(entorno)
        decision = self.decidir_accion(percepcion, t, K)
        exito = self.ejecutar_accion(decision["accion"], decision["direccion"])
        return {
            "accion": decision["accion"],
            "exito": exito,
            "razon": decision.get("razon", "")
        }

    def __repr__(self) -> str:
        """Devuelve una representación textual simplificada del agente."""
        return f"<AgenteReflejoMonstruo id={self.id} pos=({self.x},{self.y},{self.z})>"

In [2]:
# agent_robot.py
import copy
import random
from typing import Any, Dict, List, Optional, Set, Tuple

# Direcciones y rotaciones posibles en el espacio energético tridimensional
_ORIENTACIONES: Dict[str, Tuple[int, int, int]] = {
    '+X': (1, 0, 0), '-X': (-1, 0, 0),
    '+Y': (0, 1, 0), '-Y': (0, -1, 0),
    '+Z': (0, 0, 1), '-Z': (0, 0, -1)
}

# Rotación cíclica en el plano XY
_ORIENTACIONES_CICLICAS = ['+X', '+Y', '-X', '-Y']

# Clave de percepción extendida: (energómetro, roboscanner, (monstroscopio_detectado, pos_relativa), vacuscopio)
PercepcionClave = Tuple[bool, bool, Tuple[bool, Optional[str]], bool]


class AgenteRacionalRobot:
    """Agente racional tipo robot que caza monstruos en el entorno N³."""

    _TABLA_BASE: Dict[PercepcionClave, Dict[str, Any]] = {
        # Nivel 1: Energómetro activo
        (True, True, (True, "al_frente"), True): {"accion": "VACUUMATOR", "razon": "monstruo_en_celda"},
        (True, True, (True, "al_lado"), True): {"accion": "VACUUMATOR", "razon": "monstruo_en_celda"},
        (True, True, (True, "al_frente"), False): {"accion": "VACUUMATOR", "razon": "monstruo_en_celda"},
        (True, True, (True, "al_lado"), False): {"accion": "VACUUMATOR", "razon": "monstruo_en_celda"},
        (True, True, (False, None), True): {"accion": "VACUUMATOR", "razon": "monstruo_en_celda"},
        (True, True, (False, None), False): {"accion": "VACUUMATOR", "razon": "monstruo_en_celda"},
        (True, False, (True, "al_frente"), True): {"accion": "VACUUMATOR", "razon": "monstruo_en_celda"},
        (True, False, (True, "al_lado"), True): {"accion": "VACUUMATOR", "razon": "monstruo_en_celda"},
        (True, False, (True, "al_frente"), False): {"accion": "VACUUMATOR", "razon": "monstruo_en_celda"},
        (True, False, (True, "al_lado"), False): {"accion": "VACUUMATOR", "razon": "monstruo_en_celda"},
        (True, False, (False, None), True): {"accion": "VACUUMATOR", "razon": "monstruo_en_celda"},
        (True, False, (False, None), False): {"accion": "VACUUMATOR", "razon": "monstruo_en_celda"},

        # Nivel 2: Vacuscopio activo
        (False, True, (True, "al_frente"), True): {"accion": "REORIENTADOR", "param": "+90",
                                                   "razon": "obstaculo_detectado"},
        (False, True, (True, "al_lado"), True): {"accion": "REORIENTADOR", "param": "+90",
                                                 "razon": "obstaculo_detectado"},
        (False, True, (False, None), True): {"accion": "REORIENTADOR", "param": "+90", "razon": "obstaculo_detectado"},
        (False, False, (True, "al_frente"), True): {"accion": "REORIENTADOR", "param": "+90",
                                                    "razon": "obstaculo_detectado"},
        (False, False, (True, "al_lado"), True): {"accion": "REORIENTADOR", "param": "+90",
                                                  "razon": "obstaculo_detectado"},
        (False, False, (False, None), True): {"accion": "REORIENTADOR", "param": "+90", "razon": "obstaculo_detectado"},

        # Nivel 3: Robot al frente
        (False, True, (True, "al_frente"), False): {"accion": "REORIENTADOR", "param": "+90",
                                                    "razon": "robot_al_frente"},
        (False, True, (True, "al_lado"), False): {"accion": "REORIENTADOR", "param": "+90", "razon": "robot_al_frente"},
        (False, True, (False, None), False): {"accion": "REORIENTADOR", "param": "+90", "razon": "robot_al_frente"},

        # Nivel 4: Monstruo cerca
        (False, False, (True, "al_frente"), False): {"accion": "PROPULSOR", "razon": "monstruo_en_frente"},
        (False, False, (True, "al_lado"), False): {"accion": "REORIENTADOR", "razon": "alinear_con_monstruo"},

        # Nivel 5: Acción por defecto
        (False, False, (False, None), False): {"accion": "PROPULSOR", "razon": "accion_por_defecto"},
    }

    def __init__(self, id: int, x: int, y: int, z: int, orientacion: Optional[str] = None) -> None:
        """Inicializa el robot con posición, orientación y memoria independiente."""
        self.id = id
        self.x, self.y, self.z = int(x), int(y), int(z)
        self.orientacion = orientacion if orientacion in _ORIENTACIONES else random.choice(list(_ORIENTACIONES.keys()))
        self.memoria = {'historial': [], 'vacuscopio_activado': False, 'posicion_anterior': (x, y, z)}
        self.tabla_mapeo = copy.deepcopy(self._TABLA_BASE)
        self.reglas_usadas: Set[int] = set()
        self.activo = True

    # -------------------------------------------------------------------------
    # PERCEPCIÓN
    # -------------------------------------------------------------------------
    def percibir(self, robots: List[Any], monstruos: List[Any]) -> Dict[str, Any]:
        """Lee sensores locales para construir la percepción actual del entorno."""
        dx, dy, dz = _ORIENTACIONES[self.orientacion]
        frente = (self.x + dx, self.y + dy, self.z + dz)
        return {
            'giroscopio': self.orientacion,
            'energometro': any((m.x, m.y, m.z) == (self.x, self.y, self.z) for m in monstruos),
            'roboscanner': any((r.x, r.y, r.z) == frente and r.id != self.id for r in robots),
            'vacuscopio': self.memoria.get('vacuscopio_activado', False),
            'monstroscopio': self._detectar_monstruos(monstruos, dx, dy, dz),
            'posicion_anterior': self.memoria.get('posicion_anterior')
        }

    def _detectar_monstruos(self, monstruos: List[Any], dx: int, dy: int, dz: int) -> Tuple[
        bool, Optional[str], Optional[str]]:
        """Detecta monstruos al frente o a los lados, excluyendo la parte posterior."""
        atras = (-dx, -dy, -dz)
        for dir_label, (ddx, ddy, ddz) in _ORIENTACIONES.items():
            if (ddx, ddy, ddz) == atras:
                continue
            if any((m.x, m.y, m.z) == (self.x + ddx, self.y + ddy, self.z + ddz) for m in monstruos):
                return (True, "al_frente", dir_label) if (ddx, ddy, ddz) == (dx, dy, dz) else (True, "al_lado",
                                                                                               dir_label)
        return False, None, None

    # -------------------------------------------------------------------------
    # DECISIÓN Y ACCIÓN
    # -------------------------------------------------------------------------
    def decidir_accion(self, percepcion: Dict[str, Any]) -> Dict[str, Any]:
        """Selecciona la acción según la tabla percepción–acción."""
        clave = (
            percepcion["energometro"],
            percepcion["roboscanner"],
            percepcion["monstroscopio"][:2],
            percepcion["vacuscopio"]
        )
        self.memoria["vacuscopio_activado"] = False
        regla = self.tabla_mapeo.get(clave)
        if regla:
            # MÉTRICA: registrar regla usada
            if hasattr(self, "simulacion"):
                self.simulacion.metricas["reglas_usadas"].add(id(regla))
            return {"accion": regla["accion"], "param": regla.get("param", percepcion["monstroscopio"][2]),
                    "razon": regla["razon"]}
        return {"accion": "PROPULSOR", "param": percepcion["monstroscopio"][2], "razon": "accion_por_defecto"}

    def ejecutar_accion(self, accion: str, param: Optional[str], entorno: Any, monstruos: List[Any]) -> Dict[str, Any]:
        """Ejecuta el efector correspondiente (propulsor, reorientador o vacuumator)."""
        if accion == "PROPULSOR":
            return self._propulsor(entorno)
        if accion == "REORIENTADOR":
            return self._reorientador(param or "+90")
        if accion == "VACUUMATOR":
            return self._vacuumator(entorno, monstruos)
        return {"exito": False, "razon": "accion_no_reconocida", "resultado": {}}

    # -------------------------------------------------------------------------
    # EFECTORES
    # -------------------------------------------------------------------------
    def _propulsor(self, entorno: Any) -> Dict[str, Any]:
        """Avanza hacia adelante según la orientación; activa Vacuscopio si choca."""
        dx, dy, dz = _ORIENTACIONES[self.orientacion]
        nx, ny, nz = self.x + dx, self.y + dy, self.z + dz
        tipo = entorno.obtener_tipo_celda(nx, ny, nz)
        # MÉTRICA
        if hasattr(entorno, "simulacion"):
            entorno.simulacion.metricas["acciones"]["avances"] += 1
        if tipo == entorno.ZONA_LIBRE:
            self.memoria['posicion_anterior'] = (self.x, self.y, self.z)
            self.x, self.y, self.z = nx, ny, nz
            self.memoria['vacuscopio_activado'] = False
            return {"accion": "PROPULSOR", "exito": True, "razon": "avance_exitoso"}
        else:
            # MÉTRICA: colisión
            if hasattr(entorno, "simulacion"):
                entorno.simulacion.metricas["colisiones"] += 1
                if not entorno.simulacion.metricas["primer_vacuumator"]:
                    entorno.simulacion.metricas["colisiones_pre_primera_caza"] += 1
            self.memoria['vacuscopio_activado'] = True
            return {"accion": "PROPULSOR", "exito": False, "resultado": {"colision": True},
                    "razon": "colision_con_pared"}

    def _reorientador(self, sentido: str = '+90') -> Dict[str, Any]:
        """Gira 90° o se alinea a una dirección específica."""
        if hasattr(self, "simulacion"):
            self.simulacion.metricas["acciones"]["rotaciones"] += 1  # MÉTRICA
        if sentido in _ORIENTACIONES:
            self.orientacion = sentido
            return {"accion": "REORIENTADOR", "exito": True, "razon": "alineacion_directa"}
        if self.orientacion not in _ORIENTACIONES_CICLICAS:
            self.orientacion = '+X'
        i = _ORIENTACIONES_CICLICAS.index(self.orientacion)
        self.orientacion = _ORIENTACIONES_CICLICAS[(i + 1) % 4] if sentido == '+90' else _ORIENTACIONES_CICLICAS[
            (i - 1) % 4]
        return {"accion": "REORIENTADOR", "exito": True, "razon": "rotacion_lateral"}

    def _vacuumator(self, entorno: Any, monstruos: List[Any]) -> Dict[str, Any]:
        """Destruye monstruos en la celda actual y se autodestruye."""
        eliminados = [m for m in monstruos if (m.x, m.y, m.z) == (self.x, self.y, self.z)]
        for m in eliminados:
            entorno.eliminar_monstruo(m.id)
        entorno.eliminar_robot(self.id)
        entorno.grid[self.x, self.y, self.z] = entorno.ZONA_VACIA
        # MÉTRICA
        if hasattr(entorno, "simulacion"):
            entorno.simulacion.metricas["acciones"]["vacuumator"] += 1
            entorno.simulacion.metricas["monstruos_destruidos"] += len(eliminados)
            if len(eliminados) > 0:
                entorno.simulacion.metricas["primer_vacuumator"] = True
        return {"accion": "VACUUMATOR", "exito": bool(eliminados), "razon": "autodestruccion_si_exitoso"}

    # -------------------------------------------------------------------------
    # CICLO DE VIDA
    # -------------------------------------------------------------------------
    def percibir_decidir_actuar(self, t: int, entorno: Any) -> Dict[str, Any]:
        """Ejecuta un ciclo completo: percepción, decisión y acción, con evasión de bucles."""
        percepcion = self.percibir(entorno.robots, entorno.monstruos)
        decision = self.decidir_accion(percepcion)
        accion, param = decision["accion"], decision["param"]
        self.actualizar_memoria(t, percepcion, accion)
        evento = self.ejecutar_accion(accion, param, entorno, entorno.monstruos)

        bucle = self.detectar_bucle()
        if bucle:
            longitud, repeticiones = bucle
            if repeticiones >= 2:
                self._evadir_bucle(entorno)

        return {"accion": accion, "exito": evento.get("exito", False), "razon": evento.get("razon", decision["razon"])}

    # -------------------------------------------------------------------------
    # MEMORIA Y BUCLES
    # -------------------------------------------------------------------------
    def actualizar_memoria(self, t: int, percepcion: Dict[str, Any], accion: str) -> None:
        """Guarda percepciones y acciones en la memoria simbólica."""
        self.memoria['historial'].append({
            't': t,
            'p': {
                'ori': percepcion.get('giroscopio'),
                'E': percepcion.get('energometro', False),
                'R': percepcion.get('roboscanner', False),
                'M': bool(percepcion.get('monstroscopio')),
                'V': percepcion.get('vacuscopio', False),
                'pos_prev': percepcion.get('posicion_anterior'),
            },
            'a': accion
        })

    def detectar_bucle(self, min_len: int = 2, min_repeticiones: int = 2) -> Optional[Tuple[int, int]]:
        """Detecta repeticiones consecutivas de patrones de percepción–acción."""
        historial = self.memoria.get('historial', [])
        n = len(historial)
        if n < min_len * min_repeticiones:
            return None
        secuencia = [(tuple(sorted(h['p'].items())), h['a']) for h in historial]
        for l in range(min_len, n // min_repeticiones + 1):
            patron = secuencia[-l:]
            repeticiones = 1
            for i in range(2, min_repeticiones + 3):
                if n - i * l < 0:
                    break
                if patron == secuencia[-i * l:-(i - 1) * l]:
                    repeticiones += 1
                else:
                    break
            if repeticiones >= min_repeticiones:
                if hasattr(self, "simulacion"):
                    self.simulacion.metricas["bucles_detectados"] += 1  # MÉTRICA
                return l, repeticiones
        return None

    def _evadir_bucle(self, entorno: Any) -> None:
        """Cambia orientación y movimiento si se detecta un bucle conductual."""
        opuestas = {"+X": "-X", "-X": "+X", "+Y": "-Y", "-Y": "+Y", "+Z": "-Z", "-Z": "+Z"}
        historial = self.memoria.get('historial', [])[-6:]
        ultimas_oris = [h["p"]["ori"] for h in historial if "p" in h and "ori" in h["p"]]

        orientaciones_filtradas = [
                                      o for o in _ORIENTACIONES.keys()
                                      if o not in (self.orientacion,
                                                   opuestas.get(self.orientacion)) and o not in ultimas_oris
                                  ] or [
                                      o for o in _ORIENTACIONES.keys()
                                      if o not in (self.orientacion, opuestas.get(self.orientacion))
                                  ]
        nueva_dir = random.choice(orientaciones_filtradas)
        self._reorientador(nueva_dir)
        if random.random() < 0.4:
            self._propulsor(entorno)

    def exportar_historial_csv(self, carpeta: str = "resultados/historiales") -> None:
        """Exporta el historial de percepciones y acciones del robot a un CSV con fecha y hora en el nombre."""

        import csv
        import os
        from datetime import datetime

        # Crear carpeta de salida si no existe
        os.makedirs(carpeta, exist_ok=True)

        # Fecha y hora actual en formato legible
        fecha_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        nombre_archivo = f"historial_robot_{self.id}_{fecha_str}.csv"
        ruta = os.path.join(carpeta, nombre_archivo)

        # Definir columnas del CSV
        columnas = [
            "t",
            "orientacion",
            "energometro",
            "roboscanner",
            "monstroscopio",
            "vacuscopio",
            "posicion_anterior",
            "accion"
        ]

        # Crear y escribir el archivo CSV
        with open(ruta, "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=columnas)
            writer.writeheader()

            for h in self.memoria.get("historial", []):
                p = h.get("p", {})
                writer.writerow({
                    "t": h.get("t"),
                    "orientacion": p.get("ori"),
                    "energometro": p.get("E"),
                    "roboscanner": p.get("R"),
                    "monstroscopio": p.get("M"),
                    "vacuscopio": p.get("V"),
                    "posicion_anterior": p.get("pos_prev"),
                    "accion": h.get("a")
                })

        print(f"🧾 Historial del Robot {self.id} exportado en: {ruta}")

    def __repr__(self) -> str:
        """Representación simplificada del robot."""
        return f"<AgenteRacionalRobot id={self.id} pos=({self.x},{self.y},{self.z}) ori={self.orientacion}>"


In [3]:
# environment.py
import random
from typing import List, Any, Optional

import numpy as np


class EntornoOperacion:
    """Entorno tridimensional N³ donde interactúan robots y monstruos."""

    ZONA_LIBRE = 0
    ZONA_VACIA = 1

    def __init__(self, N: int = 5, Pfree: float = 0.8, Psoft: float = 0.2, seed: Optional[int] = None) -> None:
        """Inicializa el cubo energético con distribución aleatoria de zonas libres y vacías."""
        self.N = N
        self.Pfree = Pfree
        self.Psoft = Psoft

        if seed is not None:
            random.seed(seed)
            np.random.seed(seed)

        self.grid = np.zeros((N, N, N), dtype=int)
        self._generar_entorno_aleatorio()
        self.robots: List[Any] = []
        self.monstruos: List[Any] = []

    # -------------------------------------------------------------------------
    # GENERACIÓN
    # -------------------------------------------------------------------------
    def _generar_entorno_aleatorio(self) -> None:
        """Genera el entorno asignando Zonas Libres o Vacías según la proporción definida."""
        for x in range(self.N):
            for y in range(self.N):
                for z in range(self.N):
                    self.grid[x, y, z] = self.ZONA_VACIA if random.random() < self.Psoft else self.ZONA_LIBRE

        centro = self.N // 2
        self.grid[centro, centro, centro] = self.ZONA_LIBRE

        total_vacias = int(np.sum(self.grid == self.ZONA_VACIA))
        porcentaje = total_vacias / (self.N ** 3)
        print(
            f"🌍 Entorno generado ({self.N}³): {total_vacias} Zonas Vacías ({porcentaje:.1%}), "
            f"{100 - porcentaje * 100:.1f}% Zonas Libres."
        )

    # -------------------------------------------------------------------------
    # CONSULTA
    # -------------------------------------------------------------------------
    def obtener_tipo_celda(self, x: int, y: int, z: int) -> int:
        """Devuelve el tipo de zona (Libre o Vacía) en las coordenadas dadas."""
        if 0 <= x < self.N and 0 <= y < self.N and 0 <= z < self.N:
            return int(self.grid[x, y, z])
        return self.ZONA_VACIA

    # -------------------------------------------------------------------------
    # REGISTRO DE ENTIDADES
    # -------------------------------------------------------------------------
    def registrar_robot(self, robot: Any) -> bool:
        """Registra un robot si la celda es libre y no está ocupada por otro robot."""
        if self.obtener_tipo_celda(robot.x, robot.y, robot.z) == self.ZONA_VACIA:
            print(f"⚠️ Robot {robot.id} en Zona Vacía ({robot.x}, {robot.y}, {robot.z}).")
            return False
        if any((r.x, r.y, r.z) == (robot.x, robot.y, robot.z) for r in self.robots):
            print(f"⚠️ Zona ocupada por otro Robot en ({robot.x}, {robot.y}, {robot.z}).")
            return False
        self.robots.append(robot)
        return True

    def registrar_monstruo(self, monstruo: Any) -> bool:
        """Registra un monstruo si la celda es libre y no está ocupada por otro monstruo."""
        if self.obtener_tipo_celda(monstruo.x, monstruo.y, monstruo.z) == self.ZONA_VACIA:
            print(f"⚠️ Monstruo {monstruo.id} en Zona Vacía ({monstruo.x}, {monstruo.y}, {monstruo.z}).")
            return False
        if any((m.x, m.y, m.z) == (monstruo.x, monstruo.y, monstruo.z) for m in self.monstruos):
            print(f"⚠️ Zona ocupada por otro Monstruo en ({monstruo.x}, {monstruo.y}, {monstruo.z}).")
            return False
        self.monstruos.append(monstruo)
        return True

    # -------------------------------------------------------------------------
    # GESTIÓN
    # -------------------------------------------------------------------------
    def eliminar_robot(self, robot_id: int) -> None:
        """Desactiva un robot por su ID."""
        for r in self.robots:
            if r.id == robot_id and r.activo:
                r.activo = False
                break

    def eliminar_monstruo(self, monstruo_id: int) -> None:
        """Desactiva un monstruo por su ID."""
        for m in self.monstruos:
            if m.id == monstruo_id and m.activo:
                m.activo = False
                break

In [4]:
# visual_3d.py

import os
import sys
import time

import imageio.v2 as imageio
import numpy as np

# Detectar entorno y ajustar backend
IS_COLAB = "COLAB_GPU" in os.environ or "google.colab" in sys.modules
os.environ["PYOPENGL_PLATFORM"] = "egl" if IS_COLAB else "win32"

if IS_COLAB:
    try:
        from pyvirtualdisplay import Display
        display = Display(visible=0, size=(1024, 768))
        display.start()
        print("🖥️  Display virtual iniciado para ejecución headless en Colab.")
    except Exception as e:
        print("⚠️  No se pudo iniciar display virtual:", e)

from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *


class Visualizador3D:
    """Interfaz OpenGL para visualizar y avanzar automáticamente la simulación 3D."""

    def __init__(self, simulacion, delay: float = 0.2):
        self.simulacion = simulacion
        self.tick_actual = 0
        self.rot_x, self.rot_y = 25, -45
        self.zoom = -25
        self.mouse_last = None
        self.delay = delay
        self.running = True
        self.frames = []

    # ------------------------------------------------------------------
    def iniciar(self):
        """Configura la ventana, iluminación y ejecuta la simulación completa."""
        if not IS_COLAB:
            glutInit(sys.argv)
            glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGB | GLUT_DEPTH)
            glutInitWindowSize(900, 700)
            glutCreateWindow(b"Simulacion Energetica 3D")
        else:
            glutInit(sys.argv)
            glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGB | GLUT_DEPTH)
            glutInitWindowSize(900, 700)
            glutCreateWindow(b"Hidden")  # ventana “virtual” invisible

        glEnable(GL_DEPTH_TEST)
        glClearColor(0.07, 0.07, 0.07, 1.0)

        glEnable(GL_BLEND)
        glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)

        glEnable(GL_LIGHTING)
        glEnable(GL_LIGHT0)
        glEnable(GL_COLOR_MATERIAL)
        glColorMaterial(GL_FRONT_AND_BACK, GL_AMBIENT_AND_DIFFUSE)
        glLightfv(GL_LIGHT0, GL_POSITION, [0.5, 1.0, 1.0, 0.0])
        glLightfv(GL_LIGHT0, GL_AMBIENT, [0.3, 0.3, 0.3, 1.0])
        glLightfv(GL_LIGHT0, GL_DIFFUSE, [0.9, 0.9, 0.9, 1.0])

        glMatrixMode(GL_PROJECTION)
        glLoadIdentity()
        gluPerspective(45, 1.3, 0.1, 100.0)
        glMatrixMode(GL_MODELVIEW)

        print("\n⚡ Simulación automática iniciada (modo headless).")
        print("▶️  Grabando video sin mostrar la ventana...")

        while self.running:
            self._loop()

        print("✅ Simulación completada. Video guardado como simulacion_3d.mp4")

    # ------------------------------------------------------------------
    def _dibujar(self):
        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
        glLoadIdentity()
        glTranslatef(0, 0, self.zoom)
        glRotatef(self.rot_x, 1, 0, 0)
        glRotatef(self.rot_y, 0, 1, 0)
        glTranslatef(-self.simulacion.N / 2, -self.simulacion.N / 2, -self.simulacion.N / 2)

        self._dibujar_entorno()
        self._dibujar_agentes()
        self._dibujar_rejilla()

        width, height = 900, 700
        data = glReadPixels(0, 0, width, height, GL_RGB, GL_UNSIGNED_BYTE)
        image = np.frombuffer(data, dtype=np.uint8).reshape(height, width, 3)
        image = np.flipud(image)
        self.frames.append(image)

        glutSwapBuffers()

    def _dibujar_entorno(self):
        N = self.simulacion.N
        for x in range(N):
            for y in range(N):
                for z in range(N):
                    tipo = self.simulacion.entorno.grid[x, y, z]
                    if tipo == 1:
                        glEnable(GL_LIGHTING)
                        glDepthMask(GL_TRUE)
                        glColor4f(0.55, 0.55, 0.55, 1.0)
                        self._cubo(x, y, z, solid=True)
                    else:
                        glDisable(GL_LIGHTING)
                        glEnable(GL_BLEND)
                        glDepthMask(GL_FALSE)
                        glColor4f(0.2, 0.8, 0.2, 0.2)
                        self._cubo(x, y, z, solid=True)
                        glDepthMask(GL_TRUE)
                        glDisable(GL_BLEND)
                        glEnable(GL_LIGHTING)

    def _dibujar_agentes(self):
        glDisable(GL_LIGHTING)
        glDisable(GL_COLOR_MATERIAL)
        glEnable(GL_BLEND)
        glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)
        glDepthMask(GL_FALSE)

        glColor4f(1.0, 0.2, 0.2, 1.0)
        for robot in self.simulacion.entorno.robots:
            if not getattr(robot, "activo", True):
                continue
            glPushMatrix()
            glTranslatef(robot.x + 0.5, robot.y + 0.5, robot.z + 0.5)
            glScalef(0.92, 0.92, 0.92)
            glutSolidCube(1.0)
            glPopMatrix()
            if hasattr(robot, "orientacion"):
                self._dibujar_orientacion(robot)

        glColor4f(0.2, 0.5, 1.0, 1.0)
        for m in self.simulacion.entorno.monstruos:
            if not getattr(m, "activo", True):
                continue
            glPushMatrix()
            glTranslatef(m.x + 0.5, m.y + 0.5, m.z + 0.5)
            glScalef(0.92, 0.92, 0.92)
            glutSolidSphere(0.6, 20, 20)
            glPopMatrix()

        glDepthMask(GL_TRUE)
        glDisable(GL_BLEND)
        glEnable(GL_COLOR_MATERIAL)
        glEnable(GL_LIGHTING)

    def _dibujar_orientacion(self, robot):
        orientaciones = {
            "+X": (1, 0, 0), "-X": (-1, 0, 0),
            "+Y": (0, 1, 0), "-Y": (0, -1, 0),
            "+Z": (0, 0, 1), "-Z": (0, 0, -1)
        }
        if robot.orientacion not in orientaciones:
            return
        dx, dy, dz = orientaciones[robot.orientacion]
        base = (robot.x + 0.5, robot.y + 0.5, robot.z + 0.5)
        punta = (base[0] + dx * 0.6, base[1] + dy * 0.6, base[2] + dz * 0.6)
        glPushAttrib(GL_ALL_ATTRIB_BITS)
        glDisable(GL_LIGHTING)
        glColor3f(1.0, 1.0, 0.0)
        glLineWidth(3.0)
        glBegin(GL_LINES)
        glVertex3f(*base)
        glVertex3f(*punta)
        glEnd()
        glPushMatrix()
        glTranslatef(*punta)
        if dx == 1:
            glRotatef(90, 0, 1, 0)
        elif dx == -1:
            glRotatef(-90, 0, 1, 0)
        elif dy == 1:
            glRotatef(-90, 1, 0, 0)
        elif dy == -1:
            glRotatef(90, 1, 0, 0)
        elif dz == -1:
            glRotatef(180, 0, 1, 0)
        glutSolidCone(0.08, 0.2, 8, 8)
        glPopMatrix()
        glPopAttrib()

    def _cubo(self, x, y, z, solid=False):
        glPushMatrix()
        glTranslatef(x + 0.5, y + 0.5, z + 0.5)
        if solid:
            glutSolidCube(1.0)
            glColor3f(0.15, 0.15, 0.15)
            glutWireCube(1.02)
        else:
            glutWireCube(1.0)
        glPopMatrix()

    def _dibujar_rejilla(self):
        glDisable(GL_LIGHTING)
        glColor3f(0.3, 0.3, 0.3)
        glBegin(GL_LINES)
        N = self.simulacion.N
        for i in range(N + 1):
            glVertex3f(i, 0, 0)
            glVertex3f(i, N, 0)
            glVertex3f(0, i, 0)
            glVertex3f(N, i, 0)
        glEnd()
        glEnable(GL_LIGHTING)

    # ------------------------------------------------------------------
    def _loop(self):
        if self.running:
            self._tick()
            self._dibujar()
            time.sleep(self.delay)

    def _tick(self):
        if self.tick_actual >= self.simulacion.ticks:
            print("✅ Simulación completada (ticks alcanzados).")
            self._finalizar_simulacion()
            return

        for m in [m for m in self.simulacion.entorno.monstruos if getattr(m, "activo", True)]:
            m.percibir_decidir_actuar(self.tick_actual, self.simulacion.entorno, self.simulacion.K_monstruo)
        for r in [r for r in self.simulacion.entorno.robots if getattr(r, 'activo', True)]:
            r.percibir_decidir_actuar(self.tick_actual, self.simulacion.entorno)

        monstruos_activos = any(getattr(m, "activo", True) for m in self.simulacion.entorno.monstruos)
        robots_activos = any(getattr(r, "activo", True) for r in self.simulacion.entorno.robots)
        if not monstruos_activos and not robots_activos:
            print("🧟‍♂️ No quedan entidades activas. Finalizando simulación...")
            self._finalizar_simulacion()
            return

        self.rot_y += 1.0
        self.rot_x = 25 + 5 * np.sin(self.tick_actual * 0.05)
        self.tick_actual += 1

    def _finalizar_simulacion(self):
        self.running = False
        print("💾 Guardando video de la simulación...")
        if self.frames:
            imageio.mimsave("simulacion_3d.mp4", self.frames, fps=int(1 / self.delay))
            print("🎥 Video guardado como simulacion_3d.mp4")
        try:
            glutLeaveMainLoop()
        except Exception:
            sys.exit(0)

In [5]:
# simulation.py
import random
import time
from typing import Tuple

class SimulacionEnergetica:
    """Motor principal que coordina la interacción entre entorno, robots y monstruos."""

    def __init__(
            self,
            N: int = 6,
            Nrobots: int = 2,
            Nmonstruos: int = 2,
            ticks: int = 15,
            K_monstruo: int = 3,
            seed: int | None = None,
            Pfree: float = 0.8,
            Psoft: float = 0.2,
            p_movimiento: float = 0.7
    ) -> None:
        """Inicializa el entorno y crea los agentes energéticos y materiales."""
        self.N = N
        self.Nrobots = Nrobots
        self.Nmonstruos = Nmonstruos
        self.ticks = ticks
        self.K_monstruo = K_monstruo
        self.Pfree = Pfree
        self.Psoft = Psoft
        self.p_movimiento = p_movimiento
        self.seed = seed

        self.entorno = EntornoOperacion(N=N, Psoft=Psoft, Pfree=Pfree, seed=seed)
        self.entorno.simulacion = self  # vínculo circular controlado

        # Registro de métricas
        self.metricas = {
            "reglas_usadas": set(),
            "acciones": {"avances": 0, "rotaciones": 0, "vacuumator": 0},
            "colisiones": 0,
            "colisiones_pre_primera_caza": 0,
            "bucles_detectados": 0,
            "exitos_totales": 0,
            "acciones_totales": 0,
            "monstruos_destruidos": 0,
            "posiciones_iniciales": {},
            "posiciones_finales": {},
            "ticks_totales": 0,
            "primer_vacuumator": False,
            "tiempo_total": 0.0,
        }

        self._inicializar_agentes()

    # -------------------------------------------------------------------------
    # CONFIGURACIÓN INICIAL
    # -------------------------------------------------------------------------
    def _inicializar_agentes(self) -> None:
        """Crea y posiciona robots y monstruos en Zonas Libres aleatorias."""
        for i in range(self.Nrobots):
            while True:
                x, y, z = self._posicion_aleatoria()
                robot = AgenteRacionalRobot(id=i + 1, x=x, y=y, z=z)
                robot.simulacion = self  # para registrar métricas
                if self.entorno.registrar_robot(robot):
                    self.metricas["posiciones_iniciales"][f"robot_{i + 1}"] = (x, y, z)
                    break

        for i in range(self.Nmonstruos):
            while True:
                x, y, z = self._posicion_aleatoria()
                monstruo = AgenteReflejoMonstruo(id=i + 1, x=x, y=y, z=z, p_movimiento=self.p_movimiento)
                monstruo.simulacion = self
                if self.entorno.registrar_monstruo(monstruo):
                    break

    def _posicion_aleatoria(self) -> Tuple[int, int, int]:
        """Devuelve una posición aleatoria válida dentro del entorno N³."""
        return (
            random.randint(0, self.N - 1),
            random.randint(0, self.N - 1),
            random.randint(0, self.N - 1)
        )

    def ejecutar_visualizador_3d(self) -> None:
        """Ejecuta la simulación en modo 3D automático."""
        print("🧊 Iniciando simulación 3D automática...")
        vis = Visualizador3D(self, delay=0.2)
        vis.iniciar()

    # -------------------------------------------------------------------------
    # MOTOR DE SIMULACIÓN
    # -------------------------------------------------------------------------
    def ejecutar(self, delay: float = 0.0) -> None:
        """Ejecuta el ciclo energético principal de la simulación (modo silencioso)."""

        # ------------------------------------------------------------
        # ENCABEZADO INICIAL: parámetros configurables
        # ------------------------------------------------------------
        print(f"\n{'=' * 70}")
        print("⚡ SIMULACIÓN ENERGÉTICA 3D - PARÁMETROS INICIALES")
        print(f"{'=' * 70}")
        print(f"📦 Tamaño del entorno (N³): {self.N}x{self.N}x{self.N}")
        print(f"🤖 Robots racionales: {self.Nrobots}")
        print(f"👾 Monstruos reflejo: {self.Nmonstruos}")
        print(f"🔁 Ciclos totales: {self.ticks}")
        print(f"⏱️ Frecuencia de monstruos (K): {self.K_monstruo}")
        print(f"🌱 Semilla aleatoria: {self.seed}")
        print(f"🟩 Proporción zonas libres (Pfree): {self.Pfree}")
        print(f"⬛ Proporción zonas vacías (Psoft): {self.Psoft}")
        print(f"👣 Probabilidad movimiento monstruos: {self.p_movimiento}")
        print(f"{'=' * 70}\n")

        tiempo_inicio = time.perf_counter()

        # ------------------------------------------------------------
        # CICLO PRINCIPAL
        # ------------------------------------------------------------
        for t in range(self.ticks):
            # Monstruos reflejo activos
            for monstruo in [m for m in self.entorno.monstruos if getattr(m, "activo", True)]:
                monstruo.percibir_decidir_actuar(t, self.entorno, self.K_monstruo)

            # Robots racionales activos
            for robot in [r for r in self.entorno.robots if getattr(r, "activo", True)]:
                evento = robot.percibir_decidir_actuar(t, self.entorno)
                self.metricas["acciones_totales"] += 1
                if evento.get("exito", False):
                    self.metricas["exitos_totales"] += 1

            # Fin anticipado si no quedan agentes activos
            if not any(getattr(r, "activo", False) for r in self.entorno.robots) and \
                    not any(getattr(m, "activo", False) for m in self.entorno.monstruos):
                break

            time.sleep(delay)

        # ------------------------------------------------------------
        # FINALIZACIÓN Y CÁLCULOS
        # ------------------------------------------------------------
        self.metricas["tiempo_total"] = time.perf_counter() - tiempo_inicio
        self.metricas["ticks_totales"] = t + 1

        # Guardar posiciones finales
        for r in self.entorno.robots:
            self.metricas["posiciones_finales"][f"robot_{r.id}"] = (r.x, r.y, r.z)

        # ------------------------------------------------------------
        # MOSTRAR ESTADÍSTICAS
        # ------------------------------------------------------------
        self._mostrar_estadisticas()

        # ------------------------------------------------------------
        # EXPORTAR HISTORIALES DE CADA ROBOT
        # ------------------------------------------------------------
        for robot in self.entorno.robots:
            if hasattr(robot, "exportar_historial_csv"):
                robot.exportar_historial_csv()

    # -------------------------------------------------------------------------
    # ESTADÍSTICAS Y MÉTRICAS
    # -------------------------------------------------------------------------
    def _mostrar_estadisticas(self):
        """Muestra en consola las métricas detalladas de desempeño del agente."""
        m = self.metricas
        print(f"\n{'=' * 70}")
        print("📊 ESTADÍSTICAS FINALES")
        print(f"{'=' * 70}")
        print(f"Reglas usadas (distintas): {len(m['reglas_usadas'])}")
        print(f"Avances ejecutados: {m['acciones']['avances']}")
        print(f"Rotaciones ejecutadas: {m['acciones']['rotaciones']}")
        print(f"Vacuumator activado: {m['acciones']['vacuumator']}")
        print(f"Colisiones totales: {m['colisiones']}")
        print(f"Colisiones antes de primera caza: {m['colisiones_pre_primera_caza']}")
        print(f"Bucles detectados: {m['bucles_detectados']}")
        print(f"Ticks totales: {m['ticks_totales']}")
        print(f"Tiempo total de simulación: {m['tiempo_total']:.3f} s")

        # ---------------------------------------------------------------------
        # MÉTRICAS DERIVADAS
        # ---------------------------------------------------------------------
        acciones_totales = max(1, m["acciones_totales"])
        monstruos_totales = max(1, self.Nmonstruos)

        # Cálculos base
        m["tasa_colision"] = m["colisiones"] / acciones_totales
        m["porc_efectividad"] = (m["monstruos_destruidos"] / monstruos_totales) * 100
        m["complejidad"] = m["acciones_totales"] + len(m["reglas_usadas"]) + m["bucles_detectados"]
        m["tiempo_medio_ciclo"] = m["tiempo_total"] / max(1, m["ticks_totales"])

        # ---------------------------------------------------------------------
        # RACIONALIDAD (mejorada)
        # ---------------------------------------------------------------------
        # Ponderaciones (puedes ajustarlas)
        alpha, beta, lamb = 0.5, 0.3, 0.2

        md = m["monstruos_destruidos"]
        mt = monstruos_totales
        ae = m["exitos_totales"]
        at = acciones_totales
        bd = m["bucles_detectados"]

        # Cálculo de racionalidad ponderada
        m["racionalidad"] = (alpha * (md / mt)) + (beta * (ae / at)) - (lamb * (bd / at))

        # ---------------------------------------------------------------------
        # SALIDA FINAL
        # ---------------------------------------------------------------------
        print("\n🔢 MÉTRICAS DERIVADAS:")
        print(f"→ Complejidad del agente: {m['complejidad']}")
        print(
            f"→ Porcentaje de efectividad: {m['porc_efectividad']:.1f}% ({m['monstruos_destruidos']}/{self.Nmonstruos})")
        print(f"→ Tasa de colisión: {m['tasa_colision']:.3f}")
        print(f"→ Tiempo medio por ciclo: {m['tiempo_medio_ciclo']:.4f} s")
        print(f"→ Desempeño (racionalidad): {m['racionalidad']:.3f}")

        for rid, pos_final in m["posiciones_finales"].items():
            pos_ini = m["posiciones_iniciales"].get(rid)
            print(f"¿{rid} retorna a posición inicial? {'Sí' if pos_ini == pos_final else 'No'}")

        print(f"{'=' * 70}\n")

    def __repr__(self) -> str:
        """Retorna una representación textual de la configuración de la simulación."""
        return (
            f"<SimulacionEnergetica N={self.N}, Robots={len(self.entorno.robots)}, "
            f"Monstruos={len(self.entorno.monstruos)}, Ticks={self.ticks}>"
        )

In [6]:
simulacionE1 = SimulacionEnergetica(
        N=6,  # mismo tamaño
        Nrobots=2,  # igual que el intermedio
        Nmonstruos=2,  # igual que el intermedio
        ticks=100,  # un poco menos: entorno más simple
        K_monstruo=999,  # monstruos prácticamente inmóviles
        seed=42,  # misma semilla para comparabilidad
        Pfree=0.95,  # + espacios libres → menos colisiones
        Psoft=0.05,  # - obstáculos
        p_movimiento=0.05  # ~0: elimina el azar en los adversarios
    )

simulacionE1.ejecutar(delay=0.2)



🌍 Entorno generado (6³): 10 Zonas Vacías (4.6%), 95.4% Zonas Libres.

⚡ SIMULACIÓN ENERGÉTICA 3D - PARÁMETROS INICIALES
📦 Tamaño del entorno (N³): 6x6x6
🤖 Robots racionales: 2
👾 Monstruos reflejo: 2
🔁 Ciclos totales: 100
⏱️ Frecuencia de monstruos (K): 999
🌱 Semilla aleatoria: 42
🟩 Proporción zonas libres (Pfree): 0.95
⬛ Proporción zonas vacías (Psoft): 0.05
👣 Probabilidad movimiento monstruos: 0.05


📊 ESTADÍSTICAS FINALES
Reglas usadas (distintas): 7
Avances ejecutados: 64
Rotaciones ejecutadas: 11
Vacuumator activado: 2
Colisiones totales: 10
Colisiones antes de primera caza: 2
Bucles detectados: 1
Ticks totales: 70
Tiempo total de simulación: 13.858 s

🔢 MÉTRICAS DERIVADAS:
→ Complejidad del agente: 84
→ Porcentaje de efectividad: 100.0% (2/2)
→ Tasa de colisión: 0.132
→ Tiempo medio por ciclo: 0.1980 s
→ Desempeño (racionalidad): 0.758
¿robot_1 retorna a posición inicial? No
¿robot_2 retorna a posición inicial? No

🧾 Historial del Robot 1 exportado en: resultados/historiales\histo

In [7]:
simulacionE2 = SimulacionEnergetica(
        N=6,  # Tamaño del entorno cúbico (N³)
        Nrobots=2,  # Número de robots racionales
        Nmonstruos=2,  # Número de monstruos reflejo
        ticks=150,  # Ciclos totales de ejecución de la simulación
        K_monstruo=3,  # Frecuencia de acción de los monstruos (cada K ciclos)
        seed=42,  # Semilla aleatoria para reproducibilidad
        Pfree=0.8,  # Proporción de zonas libres (transitables)
        Psoft=0.2,  # Proporción de zonas vacías (obstáculos)
        p_movimiento=0.7  # Probabilidad de movimiento de cada monstruo
    )

simulacionE2.ejecutar(delay=0.2)

🌍 Entorno generado (6³): 40 Zonas Vacías (18.5%), 81.5% Zonas Libres.
⚠️ Robot 2 en Zona Vacía (1, 1, 2).

⚡ SIMULACIÓN ENERGÉTICA 3D - PARÁMETROS INICIALES
📦 Tamaño del entorno (N³): 6x6x6
🤖 Robots racionales: 2
👾 Monstruos reflejo: 2
🔁 Ciclos totales: 150
⏱️ Frecuencia de monstruos (K): 3
🌱 Semilla aleatoria: 42
🟩 Proporción zonas libres (Pfree): 0.8
⬛ Proporción zonas vacías (Psoft): 0.2
👣 Probabilidad movimiento monstruos: 0.7


📊 ESTADÍSTICAS FINALES
Reglas usadas (distintas): 13
Avances ejecutados: 74
Rotaciones ejecutadas: 53
Vacuumator activado: 2
Colisiones totales: 22
Colisiones antes de primera caza: 11
Bucles detectados: 6
Ticks totales: 96
Tiempo total de simulación: 19.102 s

🔢 MÉTRICAS DERIVADAS:
→ Complejidad del agente: 139
→ Porcentaje de efectividad: 100.0% (2/2)
→ Tasa de colisión: 0.183
→ Tiempo medio por ciclo: 0.1990 s
→ Desempeño (racionalidad): 0.737
¿robot_1 retorna a posición inicial? No
¿robot_2 retorna a posición inicial? No

🧾 Historial del Robot 1 exporta

In [8]:
simulacionE3 = SimulacionEnergetica(
        N=6,  # mismo tamaño
        Nrobots=2,  # igual que el intermedio
        Nmonstruos=2,  # igual que el intermedio
        ticks=200,  # más largo: más interacción dinámica
        K_monstruo=2,  # monstruos actúan con mayor frecuencia
        seed=42,  # misma semilla para comparabilidad
        Pfree=0.7,  # un poco menos de libres que el intermedio
        Psoft=0.3,  # un poco más de obstáculos (sin ser extremo)
        p_movimiento=0.9  # alta movilidad adversaria
)

# Ejecución de la simulación
simulacionE3.ejecutar(delay=0.2)
#simulacion.ejecutar_manual_3d()

🌍 Entorno generado (6³): 74 Zonas Vacías (34.3%), 65.7% Zonas Libres.
⚠️ Robot 2 en Zona Vacía (1, 1, 2).

⚡ SIMULACIÓN ENERGÉTICA 3D - PARÁMETROS INICIALES
📦 Tamaño del entorno (N³): 6x6x6
🤖 Robots racionales: 2
👾 Monstruos reflejo: 2
🔁 Ciclos totales: 200
⏱️ Frecuencia de monstruos (K): 2
🌱 Semilla aleatoria: 42
🟩 Proporción zonas libres (Pfree): 0.7
⬛ Proporción zonas vacías (Psoft): 0.3
👣 Probabilidad movimiento monstruos: 0.9


📊 ESTADÍSTICAS FINALES
Reglas usadas (distintas): 12
Avances ejecutados: 199
Rotaciones ejecutadas: 85
Vacuumator activado: 2
Colisiones totales: 59
Colisiones antes de primera caza: 54
Bucles detectados: 9
Ticks totales: 152
Tiempo total de simulación: 32.172 s

🔢 MÉTRICAS DERIVADAS:
→ Complejidad del agente: 295
→ Porcentaje de efectividad: 100.0% (2/2)
→ Tasa de colisión: 0.215
→ Tiempo medio por ciclo: 0.2117 s
→ Desempeño (racionalidad): 0.730
¿robot_1 retorna a posición inicial? No
¿robot_2 retorna a posición inicial? No

🧾 Historial del Robot 1 expor

In [9]:
simulacion = SimulacionEnergetica(
    N=6,  # Tamaño del entorno cúbico (N³)
    Nrobots=2,  # Número de robots racionales
    Nmonstruos=2,  # Número de monstruos reflejo
    ticks=150,  # Ciclos totales de ejecución de la simulación
    K_monstruo=3,  # Frecuencia de acción de los monstruos (cada K ciclos)
    seed=42,  # Semilla aleatoria para reproducibilidad
    Pfree=0.8,  # Proporción de zonas libres (transitables)
    Psoft=0.2,  # Proporción de zonas vacías (obstáculos)
    p_movimiento=0.7  # Probabilidad de movimiento de cada monstruo
)

simulacion.ejecutar_visualizador_3d()

🌍 Entorno generado (6³): 40 Zonas Vacías (18.5%), 81.5% Zonas Libres.
⚠️ Robot 2 en Zona Vacía (1, 1, 2).
🧊 Iniciando simulación 3D automática...

⚡ Simulación automática iniciada (modo headless).
▶️  Grabando video sin mostrar la ventana...
🧟‍♂️ No quedan entidades activas. Finalizando simulación...
💾 Guardando video de la simulación...


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (900, 700) to (912, 704) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


🎥 Video guardado como simulacion_3d.mp4
✅ Simulación completada. Video guardado como simulacion_3d.mp4


In [10]:
from IPython.display import Video
Video("simulacion_3d.mp4", embed=True)